# Amazon Fine Food Reviews Analysis


Data Source: https://www.kaggle.com/snap/amazon-fine-food-reviews <br>

EDA: https://nycdatascience.com/blog/student-works/amazon-fine-foods-visualization/


The Amazon Fine Food Reviews dataset consists of reviews of fine foods from Amazon.<br>

Number of reviews: 568,454<br>
Number of users: 256,059<br>
Number of products: 74,258<br>
Timespan: Oct 1999 - Oct 2012<br>
Number of Attributes/Columns in data: 10 

Attribute Information:

1. Id
2. ProductId - unique identifier for the product
3. UserId - unqiue identifier for the user
4. ProfileName
5. HelpfulnessNumerator - number of users who found the review helpful
6. HelpfulnessDenominator - number of users who indicated whether they found the review helpful or not
7. Score - rating between 1 and 5
8. Time - timestamp for the review
9. Summary - brief summary of the review
10. Text - text of the review


#### Objective:
Given a review, determine whether the review is positive (rating of 4 or 5) or negative (rating of 1 or 2).

<br>
[Q] How to determine if a review is positive or negative?<br>
<br> 
[Ans] We could use Score/Rating. A rating of 4 or 5 can be cosnidered as a positive review. A rating of 1 or 2 can be considered as negative one. A review of rating 3 is considered nuetral and such reviews are ignored from our analysis. This is an approximate and proxy way of determining the polarity (positivity/negativity) of a review.




# [1]. Reading Data

## [1.1] Loading the data

The dataset is available in two forms
1. .csv file
2. SQLite Database

In order to load the data, We have used the SQLITE dataset as it is easier to query the data and visualise the data efficiently.
<br> 

Here as we only want to get the global sentiment of the recommendations (positive or negative), we will purposefully ignore all Scores equal to 3. If the score is above 3, then the recommendation wil be set to "positive". Otherwise, it will be set to "negative".

In [34]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from tqdm import tqdm
import os
import numpy
import pandas as pd
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

In [2]:
# using SQLite Table to read data.
con = sqlite3.connect('C:\\Users\\admin\\Downloads\\AppliedAi\\Amazon Fine Food Reviews\\database.sqlite') 

# filtering only positive and negative reviews i.e. 
# not taking into consideration those reviews with Score=3
# SELECT * FROM Reviews WHERE Score != 3 LIMIT 500000, will give top 500000 data points
# you can change the number to any other number based on your computing power

# filtered_data = pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score != 3 LIMIT 500000""", con) 
# for tsne assignment you can take 5k data points

filtered_data = pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score != 3 LIMIT 50000""", con) 

# Give reviews with Score>3 a positive rating(1), and reviews with a score<3 a negative rating(0).
def partition(x):
    if x < 3:
        return 0
    return 1

#changing reviews with score less than 3 to be positive and vice-versa
actualScore = filtered_data['Score']
positiveNegative = actualScore.map(partition) 
filtered_data['Score'] = positiveNegative
print("Number of data points in our data", filtered_data.shape)
filtered_data.head(3)

Number of data points in our data (50000, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...


In [3]:
display = pd.read_sql_query("""
SELECT UserId, ProductId, ProfileName, Time, Score, Text, COUNT(*)
FROM Reviews
GROUP BY UserId
HAVING COUNT(*)>1
""", con)

In [4]:
print(display.shape)
display.head()

(80668, 7)


,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
0,#oc-R115TNMSPFT9I7,B005ZBZLT4,Breyton,1331510400,2,Overall its just OK when considering the price...,2
1,#oc-R11D9D7SHXIJB9,B005HG9ESG,"Louis E. Emory ""hoppy""",1342396800,5,"My wife has recurring extreme muscle spasms, u...",3
2,#oc-R11DNU2NBKQ23Z,B005ZBZLT4,Kim Cieszykowski,1348531200,1,This coffee is horrible and unfortunately not ...,2
3,#oc-R11O5J5ZVQE25C,B005HG9ESG,Penguin Chick,1346889600,5,This will be the bottle that you grab from the...,3
4,#oc-R12KPBODL2B5ZD,B007OSBEV0,Christopher P. Presta,1348617600,1,I didnt like this coffee. Instead of telling y...,2


In [5]:
display[display['UserId']=='AZY10LLTJ71NX']

,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
80638,AZY10LLTJ71NX,B001ATMQK2,"undertheshrine ""undertheshrine""",1296691200,5,I bought this 6 pack because for the price tha...,5


In [6]:
display['COUNT(*)'].sum()

393063

#  [2] Exploratory Data Analysis

## [2.1] Data Cleaning: Deduplication

It is observed (as shown in the table below) that the reviews data had many duplicate entries. Hence it was necessary to remove duplicates in order to get unbiased results for the analysis of the data.  Following is an example:

In [7]:
display= pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND UserId="AR5J8UI46CURR"
ORDER BY ProductID
""", con)
display.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


As it can be seen above that same user has multiple reviews with same values for HelpfulnessNumerator, HelpfulnessDenominator, Score, Time, Summary and Text and on doing analysis it was found that <br>
<br> 
ProductId=B000HDOPZG was Loacker Quadratini Vanilla Wafer Cookies, 8.82-Ounce Packages (Pack of 8)<br>
<br> 
ProductId=B000HDL1RQ was Loacker Quadratini Lemon Wafer Cookies, 8.82-Ounce Packages (Pack of 8) and so on<br>

It was inferred after analysis that reviews with same parameters other than ProductId belonged to the same product just having different flavour or quantity. Hence in order to reduce redundancy it was decided to eliminate the rows having same parameters.<br>

The method used for the same was that we first sort the data according to ProductId and then just keep the first similar product review and delelte the others. for eg. in the above just the review for ProductId=B000HDL1RQ remains. This method ensures that there is only one representative for each product and deduplication without sorting would lead to possibility of different representatives still existing for the same product.

In [8]:
#Sorting data according to ProductId in ascending order
sorted_data=filtered_data.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

In [9]:
#Deduplication of entries
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
final.shape

(46072, 10)

In [10]:
#Checking to see how much % of data still remains
(final['Id'].size*1.0)/(filtered_data['Id'].size*1.0)*100

92.144

<b>Observation:-</b> It was also seen that in two rows given below the value of HelpfulnessNumerator is greater than HelpfulnessDenominator which is not practically possible hence these two rows too are removed from calcualtions

In [11]:
display= pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND Id=44737 OR Id=64422
ORDER BY ProductID
""", con)

display.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
1,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


In [12]:
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]

In [13]:
#Before starting the next phase of preprocessing lets see the number of entries left
print(final.shape)

#How many positive and negative reviews are present in our dataset?
final['Score'].value_counts()

(46071, 10)


1    38479
0     7592
Name: Score, dtype: int64

#  [3] Preprocessing

## [3.1].  Preprocessing Review Text

Now that we have finished deduplication our data requires some preprocessing before we go on further with analysis and making the prediction model.

Hence in the Preprocessing phase we do the following in the order below:-

1. Begin by removing the html tags
2. Remove any punctuations or limited set of special characters like , or . or # etc.
3. Check if the word is made up of english letters and is not alpha-numeric
4. Check to see if the length of the word is greater than 2 (as it was researched that there is no adjective in 2-letters)
5. Convert the word to lowercase
6. Remove Stopwords
7. Finally Snowball Stemming the word (it was obsereved to be better than Porter Stemming)<br>

After which we collect the words used to describe positive and negative reviews

In [14]:
# printing some random reviews
sent_0 = final['Text'].values[0]
print(sent_0)
print("="*50)

sent_1000 = final['Text'].values[1000]
print(sent_1000)
print("="*50)

sent_1500 = final['Text'].values[1500]
print(sent_1500)
print("="*50)

sent_4900 = final['Text'].values[4900]
print(sent_4900)
print("="*50)

My dogs loves this chicken but its a product from China, so we wont be buying it anymore.  Its very hard to find any chicken products made in the USA but they are out there, but this one isnt.  Its too bad too because its a good product but I wont take any chances till they know what is going on with the china imports.
this is yummy, easy and unusual. it makes a quick, delicous pie, crisp or cobbler. home made is better, but a heck of a lot more work. this is great to have on hand for last minute dessert needs where you really want to impress wih your creativity in cooking! recommended.
Great flavor, low in calories, high in nutrients, high in protein! Usually protein powders are high priced and high in calories, this one is a great bargain and tastes great, I highly recommend for the lady gym rats, probably not "macho" enough for guys since it is soy based...
For those of you wanting a high-quality, yet affordable green tea, you should definitely give this one a try. Let me first star

In [15]:
# remove urls from text python: https://stackoverflow.com/a/40823105/4084039
sent_0 = re.sub(r"http\S+", "", sent_0)
sent_1000 = re.sub(r"http\S+", "", sent_1000)
sent_150 = re.sub(r"http\S+", "", sent_1500)
sent_4900 = re.sub(r"http\S+", "", sent_4900)

print(sent_0)

My dogs loves this chicken but its a product from China, so we wont be buying it anymore.  Its very hard to find any chicken products made in the USA but they are out there, but this one isnt.  Its too bad too because its a good product but I wont take any chances till they know what is going on with the china imports.


In [16]:
# https://stackoverflow.com/questions/16206380/python-beautifulsoup-how-to-remove-all-tags-from-an-element
from bs4 import BeautifulSoup

soup = BeautifulSoup(sent_0, 'lxml')
text = soup.get_text()
print(text)
print("="*50)

soup = BeautifulSoup(sent_1000, 'lxml')
text = soup.get_text()
print(text)
print("="*50)

soup = BeautifulSoup(sent_1500, 'lxml')
text = soup.get_text()
print(text)
print("="*50)

soup = BeautifulSoup(sent_4900, 'lxml')
text = soup.get_text()
print(text)

My dogs loves this chicken but its a product from China, so we wont be buying it anymore.  Its very hard to find any chicken products made in the USA but they are out there, but this one isnt.  Its too bad too because its a good product but I wont take any chances till they know what is going on with the china imports.
this is yummy, easy and unusual. it makes a quick, delicous pie, crisp or cobbler. home made is better, but a heck of a lot more work. this is great to have on hand for last minute dessert needs where you really want to impress wih your creativity in cooking! recommended.
Great flavor, low in calories, high in nutrients, high in protein! Usually protein powders are high priced and high in calories, this one is a great bargain and tastes great, I highly recommend for the lady gym rats, probably not "macho" enough for guys since it is soy based...
For those of you wanting a high-quality, yet affordable green tea, you should definitely give this one a try. Let me first star

In [17]:
# https://stackoverflow.com/a/47091490/4084039
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [18]:
sent_1500 = decontracted(sent_1500)
print(sent_1500)
print("="*50)

Great flavor, low in calories, high in nutrients, high in protein! Usually protein powders are high priced and high in calories, this one is a great bargain and tastes great, I highly recommend for the lady gym rats, probably not "macho" enough for guys since it is soy based...


In [19]:
#remove words with numbers python: https://stackoverflow.com/a/18082370/4084039
sent_0 = re.sub("\S*\d\S*", "", sent_0).strip()
print(sent_0)

My dogs loves this chicken but its a product from China, so we wont be buying it anymore.  Its very hard to find any chicken products made in the USA but they are out there, but this one isnt.  Its too bad too because its a good product but I wont take any chances till they know what is going on with the china imports.


In [20]:
#remove spacial character: https://stackoverflow.com/a/5843547/4084039
sent_1500 = re.sub('[^A-Za-z0-9]+', ' ', sent_1500)
print(sent_1500)

Great flavor low in calories high in nutrients high in protein Usually protein powders are high priced and high in calories this one is a great bargain and tastes great I highly recommend for the lady gym rats probably not macho enough for guys since it is soy based 


In [21]:
# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not'
# <br /><br /> ==> after the above steps, we are getting "br br"
# we are including them into stop words list
# instead of <br /> if we have <br/> these tags would have revmoved in the 1st step

stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [22]:
# Combining all the above stundents 
from tqdm import tqdm
preprocessed_reviews = []
# tqdm is for printing the status bar
for sentance in tqdm(final['Text'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_reviews.append(sentance.strip())

100%|██████████████████████████████████| 46071/46071 [00:24<00:00, 1918.67it/s]


# Splitting into train and test

In [29]:
from sklearn.model_selection import train_test_split
Score=final.Score
preprocessed_reviews=np.array(preprocessed_reviews)
X_train, X_test, y_train, y_test = train_test_split(preprocessed_reviews, Score, test_size=0.30,random_state=0)

In [30]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(32249,) (32249,)
(13822,) (13822,)


In [31]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)

In [39]:
X_test = tokenizer.texts_to_sequences(X_test)

In [32]:
print(X_train[45])
print(type(X_train[45]))
print(len(X_train[45]))

[3, 1, 2450, 423, 732, 62, 432, 613, 455, 164, 2, 11, 289, 39, 259, 1415, 239, 259, 6, 148, 130, 476, 14, 1627, 7, 56]
<class 'list'>
26


In [41]:
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
print(X_train.shape)
print(X_train[45])

(32249, 500)
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    

# Model 1

In [42]:
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(5000, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


In [44]:
model.fit(X_train, y_train, nb_epoch=5, batch_size=64)
scores = model.evaluate(X_test, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/5


13056/32249 [===========>..................] - ETA: 4:56 - loss: 0.4928 - acc: 0.796 - ETA: 5:03 - loss: 0.4919 - acc: 0.796 - ETA: 5:06 - loss: 0.4597 - acc: 0.817 - ETA: 5:05 - loss: 0.4557 - acc: 0.820 - ETA: 5:04 - loss: 0.4450 - acc: 0.828 - ETA: 4:58 - loss: 0.4535 - acc: 0.820 - ETA: 4:55 - loss: 0.4554 - acc: 0.819 - ETA: 4:53 - loss: 0.4667 - acc: 0.810 - ETA: 4:50 - loss: 0.4752 - acc: 0.803 - ETA: 4:48 - loss: 0.4636 - acc: 0.812 - ETA: 4:46 - loss: 0.4676 - acc: 0.809 - ETA: 4:44 - loss: 0.4642 - acc: 0.809 - ETA: 4:43 - loss: 0.4682 - acc: 0.806 - ETA: 4:41 - loss: 0.4623 - acc: 0.810 - ETA: 4:40 - loss: 0.4590 - acc: 0.811 - ETA: 4:38 - loss: 0.4585 - acc: 0.811 - ETA: 4:35 - loss: 0.4575 - acc: 0.810 - ETA: 4:32 - loss: 0.4555 - acc: 0.810 - ETA: 4:32 - loss: 0.4481 - acc: 0.815 - ETA: 4:31 - loss: 0.4450 - acc: 0.817 - ETA: 4:31 - loss: 0.4406 - acc: 0.819 - ETA: 4:30 - loss: 0.4417 - acc: 0.818 - ETA: 4:29 - loss: 0.4388 - acc: 0.819 - ETA: 4:29 - loss: 0.4411 - acc: 0

26176/32249 [=======================>......] - ETA: 2:52 - loss: 0.2994 - acc: 0.876 - ETA: 2:51 - loss: 0.2990 - acc: 0.876 - ETA: 2:51 - loss: 0.2987 - acc: 0.876 - ETA: 2:50 - loss: 0.2984 - acc: 0.876 - ETA: 2:49 - loss: 0.2984 - acc: 0.876 - ETA: 2:49 - loss: 0.2982 - acc: 0.876 - ETA: 2:48 - loss: 0.2978 - acc: 0.876 - ETA: 2:48 - loss: 0.2973 - acc: 0.877 - ETA: 2:47 - loss: 0.2965 - acc: 0.877 - ETA: 2:46 - loss: 0.2959 - acc: 0.877 - ETA: 2:46 - loss: 0.2955 - acc: 0.877 - ETA: 2:45 - loss: 0.2957 - acc: 0.877 - ETA: 2:45 - loss: 0.2958 - acc: 0.877 - ETA: 2:44 - loss: 0.2955 - acc: 0.877 - ETA: 2:43 - loss: 0.2953 - acc: 0.877 - ETA: 2:43 - loss: 0.2956 - acc: 0.877 - ETA: 2:42 - loss: 0.2956 - acc: 0.878 - ETA: 2:42 - loss: 0.2956 - acc: 0.878 - ETA: 2:41 - loss: 0.2956 - acc: 0.877 - ETA: 2:41 - loss: 0.2952 - acc: 0.878 - ETA: 2:40 - loss: 0.2949 - acc: 0.878 - ETA: 2:40 - loss: 0.2945 - acc: 0.878 - ETA: 2:39 - loss: 0.2943 - acc: 0.878 - ETA: 2:38 - loss: 0.2938 - acc: 0

32249/32249 [==============================] - ETA: 54s - loss: 0.2627 - acc: 0.89 - ETA: 53s - loss: 0.2626 - acc: 0.89 - ETA: 53s - loss: 0.2625 - acc: 0.89 - ETA: 52s - loss: 0.2625 - acc: 0.89 - ETA: 51s - loss: 0.2622 - acc: 0.89 - ETA: 51s - loss: 0.2624 - acc: 0.89 - ETA: 50s - loss: 0.2621 - acc: 0.89 - ETA: 50s - loss: 0.2623 - acc: 0.89 - ETA: 49s - loss: 0.2624 - acc: 0.89 - ETA: 49s - loss: 0.2622 - acc: 0.89 - ETA: 48s - loss: 0.2624 - acc: 0.89 - ETA: 47s - loss: 0.2621 - acc: 0.89 - ETA: 47s - loss: 0.2623 - acc: 0.89 - ETA: 46s - loss: 0.2623 - acc: 0.89 - ETA: 46s - loss: 0.2622 - acc: 0.89 - ETA: 45s - loss: 0.2618 - acc: 0.89 - ETA: 45s - loss: 0.2617 - acc: 0.89 - ETA: 44s - loss: 0.2615 - acc: 0.89 - ETA: 43s - loss: 0.2615 - acc: 0.89 - ETA: 43s - loss: 0.2615 - acc: 0.89 - ETA: 42s - loss: 0.2613 - acc: 0.89 - ETA: 42s - loss: 0.2611 - acc: 0.89 - ETA: 41s - loss: 0.2609 - acc: 0.89 - ETA: 41s - loss: 0.2611 - acc: 0.89 - ETA: 40s - loss: 0.2611 - acc: 0.89 - ETA

13056/32249 [===========>..................] - ETA: 4:49 - loss: 0.1948 - acc: 0.937 - ETA: 4:48 - loss: 0.1633 - acc: 0.953 - ETA: 4:49 - loss: 0.1422 - acc: 0.958 - ETA: 4:55 - loss: 0.1414 - acc: 0.953 - ETA: 4:56 - loss: 0.1392 - acc: 0.950 - ETA: 4:53 - loss: 0.1399 - acc: 0.945 - ETA: 4:53 - loss: 0.1327 - acc: 0.950 - ETA: 4:53 - loss: 0.1361 - acc: 0.951 - ETA: 4:52 - loss: 0.1420 - acc: 0.947 - ETA: 4:50 - loss: 0.1479 - acc: 0.945 - ETA: 4:49 - loss: 0.1560 - acc: 0.943 - ETA: 4:48 - loss: 0.1626 - acc: 0.944 - ETA: 4:48 - loss: 0.1634 - acc: 0.943 - ETA: 4:46 - loss: 0.1740 - acc: 0.940 - ETA: 4:42 - loss: 0.1739 - acc: 0.940 - ETA: 4:40 - loss: 0.1711 - acc: 0.941 - ETA: 4:40 - loss: 0.1681 - acc: 0.942 - ETA: 4:39 - loss: 0.1697 - acc: 0.941 - ETA: 4:39 - loss: 0.1652 - acc: 0.942 - ETA: 4:38 - loss: 0.1706 - acc: 0.941 - ETA: 4:38 - loss: 0.1692 - acc: 0.942 - ETA: 4:38 - loss: 0.1688 - acc: 0.943 - ETA: 4:39 - loss: 0.1672 - acc: 0.943 - ETA: 4:40 - loss: 0.1644 - acc: 0

...

26112/32249 [=======================>......] - ETA: 2:55 - loss: 0.1187 - acc: 0.956 - ETA: 2:54 - loss: 0.1188 - acc: 0.956 - ETA: 2:53 - loss: 0.1192 - acc: 0.956 - ETA: 2:53 - loss: 0.1200 - acc: 0.956 - ETA: 2:52 - loss: 0.1202 - acc: 0.956 - ETA: 2:52 - loss: 0.1204 - acc: 0.956 - ETA: 2:51 - loss: 0.1203 - acc: 0.955 - ETA: 2:51 - loss: 0.1211 - acc: 0.955 - ETA: 2:50 - loss: 0.1207 - acc: 0.955 - ETA: 2:49 - loss: 0.1210 - acc: 0.955 - ETA: 2:49 - loss: 0.1209 - acc: 0.955 - ETA: 2:48 - loss: 0.1211 - acc: 0.955 - ETA: 2:48 - loss: 0.1211 - acc: 0.955 - ETA: 2:47 - loss: 0.1211 - acc: 0.955 - ETA: 2:47 - loss: 0.1210 - acc: 0.955 - ETA: 2:46 - loss: 0.1212 - acc: 0.955 - ETA: 2:45 - loss: 0.1211 - acc: 0.955 - ETA: 2:45 - loss: 0.1210 - acc: 0.955 - ETA: 2:44 - loss: 0.1211 - acc: 0.955 - ETA: 2:43 - loss: 0.1214 - acc: 0.955 - ETA: 2:43 - loss: 0.1216 - acc: 0.955 - ETA: 2:42 - loss: 0.1215 - acc: 0.955 - ETA: 2:42 - loss: 0.1216 - acc: 0.955 - ETA: 2:41 - loss: 0.1215 - acc: 0

32249/32249 [==============================] - ETA: 55s - loss: 0.1272 - acc: 0.95 - ETA: 54s - loss: 0.1274 - acc: 0.95 - ETA: 54s - loss: 0.1273 - acc: 0.95 - ETA: 53s - loss: 0.1272 - acc: 0.95 - ETA: 53s - loss: 0.1272 - acc: 0.95 - ETA: 52s - loss: 0.1272 - acc: 0.95 - ETA: 51s - loss: 0.1274 - acc: 0.95 - ETA: 51s - loss: 0.1275 - acc: 0.95 - ETA: 50s - loss: 0.1275 - acc: 0.95 - ETA: 50s - loss: 0.1276 - acc: 0.95 - ETA: 49s - loss: 0.1278 - acc: 0.95 - ETA: 49s - loss: 0.1277 - acc: 0.95 - ETA: 48s - loss: 0.1276 - acc: 0.95 - ETA: 47s - loss: 0.1279 - acc: 0.95 - ETA: 47s - loss: 0.1277 - acc: 0.95 - ETA: 46s - loss: 0.1276 - acc: 0.95 - ETA: 46s - loss: 0.1279 - acc: 0.95 - ETA: 45s - loss: 0.1281 - acc: 0.95 - ETA: 44s - loss: 0.1285 - acc: 0.95 - ETA: 44s - loss: 0.1284 - acc: 0.95 - ETA: 43s - loss: 0.1287 - acc: 0.95 - ETA: 43s - loss: 0.1286 - acc: 0.95 - ETA: 42s - loss: 0.1286 - acc: 0.95 - ETA: 42s - loss: 0.1288 - acc: 0.95 - ETA: 41s - loss: 0.1288 - acc: 0.95 - ETA

13056/32249 [===========>..................] - ETA: 4:51 - loss: 0.0580 - acc: 0.984 - ETA: 4:49 - loss: 0.0533 - acc: 0.992 - ETA: 4:47 - loss: 0.0576 - acc: 0.984 - ETA: 4:47 - loss: 0.0781 - acc: 0.976 - ETA: 4:46 - loss: 0.0765 - acc: 0.978 - ETA: 4:45 - loss: 0.0879 - acc: 0.971 - ETA: 4:45 - loss: 0.0875 - acc: 0.971 - ETA: 4:48 - loss: 0.0929 - acc: 0.966 - ETA: 4:50 - loss: 0.0913 - acc: 0.967 - ETA: 4:52 - loss: 0.0859 - acc: 0.970 - ETA: 4:51 - loss: 0.0860 - acc: 0.970 - ETA: 4:50 - loss: 0.0898 - acc: 0.967 - ETA: 4:47 - loss: 0.0849 - acc: 0.970 - ETA: 4:44 - loss: 0.0860 - acc: 0.968 - ETA: 4:41 - loss: 0.0839 - acc: 0.969 - ETA: 4:41 - loss: 0.0827 - acc: 0.970 - ETA: 4:40 - loss: 0.0842 - acc: 0.970 - ETA: 4:40 - loss: 0.0841 - acc: 0.970 - ETA: 4:39 - loss: 0.0865 - acc: 0.969 - ETA: 4:38 - loss: 0.0860 - acc: 0.970 - ETA: 4:38 - loss: 0.0855 - acc: 0.971 - ETA: 4:37 - loss: 0.0835 - acc: 0.972 - ETA: 4:37 - loss: 0.0846 - acc: 0.972 - ETA: 4:36 - loss: 0.0843 - acc: 0

26112/32249 [=======================>......] - ETA: 2:55 - loss: 0.1021 - acc: 0.964 - ETA: 2:54 - loss: 0.1017 - acc: 0.964 - ETA: 2:53 - loss: 0.1019 - acc: 0.964 - ETA: 2:53 - loss: 0.1020 - acc: 0.964 - ETA: 2:52 - loss: 0.1019 - acc: 0.964 - ETA: 2:52 - loss: 0.1018 - acc: 0.964 - ETA: 2:51 - loss: 0.1020 - acc: 0.964 - ETA: 2:50 - loss: 0.1021 - acc: 0.964 - ETA: 2:50 - loss: 0.1019 - acc: 0.964 - ETA: 2:49 - loss: 0.1018 - acc: 0.964 - ETA: 2:49 - loss: 0.1022 - acc: 0.964 - ETA: 2:48 - loss: 0.1021 - acc: 0.964 - ETA: 2:48 - loss: 0.1025 - acc: 0.964 - ETA: 2:47 - loss: 0.1023 - acc: 0.964 - ETA: 2:46 - loss: 0.1020 - acc: 0.964 - ETA: 2:46 - loss: 0.1018 - acc: 0.964 - ETA: 2:45 - loss: 0.1019 - acc: 0.964 - ETA: 2:45 - loss: 0.1019 - acc: 0.964 - ETA: 2:44 - loss: 0.1020 - acc: 0.964 - ETA: 2:43 - loss: 0.1019 - acc: 0.964 - ETA: 2:43 - loss: 0.1019 - acc: 0.964 - ETA: 2:42 - loss: 0.1016 - acc: 0.964 - ETA: 2:42 - loss: 0.1018 - acc: 0.964 - ETA: 2:41 - loss: 0.1016 - acc: 0

13822/13822 [==============================] - ETA: 2: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 59s - ETA: 55 - ETA: 52 - ETA: 49 - ETA: 48 - ETA: 46 - ETA: 45 - ETA: 44 - ETA: 43 - ETA: 43 - ETA: 42 - ETA: 41 - ETA: 41 - ETA: 41 - ETA: 40 - ETA: 40 - ETA: 39 - ETA: 39 - ETA: 39 - ETA: 39 - ETA: 38 - ETA: 38 - ETA: 38 - ETA: 38 - ETA: 38 - ETA: 37 - ETA: 37 - ETA: 37 - ETA: 37 - ETA: 37 - ETA: 37 - ETA: 36 - ETA: 36 - ETA: 36 - ETA: 36 - ETA: 36 - ETA: 36 - ETA: 36 - ETA: 35 - ETA: 35 - ETA: 35 - ETA: 35 - ETA: 35 - ETA: 35 - ETA: 35 - ETA: 35 - ETA: 35 - ETA: 34 - ETA: 34 - ETA: 34 - ETA: 34 - ETA: 34 - ETA: 34 - ETA: 34 - ETA: 34 - ETA: 34 - ETA: 33 - ETA: 33 - ETA: 33 - ETA: 33 - ETA: 33 - ETA: 33 - ETA: 33 - ETA: 33 - ETA: 33 - ETA: 32 - ETA: 32 - ETA: 32 - ETA: 32 - ETA: 32 - ETA: 32 - ETA: 32 - ETA: 32 - ETA: 32 - ETA: 31 - ETA: 31 - ETA: 31 - ETA: 31 - ETA: 31 - ETA: 31 - ETA: 31 - ETA: 31 - ETA: 31 - ETA: 31 - ETA: 31 - ETA: 30 - ETA: 30 - ETA: 30 - ETA: 30 - ETA: 30 - ETA: 30 - ET

# Model 2

In [46]:
embedding_vecor_length = 32
model1 = Sequential()
model1.add(Embedding(5000, embedding_vecor_length, input_length=max_review_length))
model1.add(LSTM(100, return_sequences=True))
model1.add(LSTM(100))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model1.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 500, 100)          53200     
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 293,701
Trainable params: 293,701
Non-trainable params: 0
_________________________________________________________________
None


In [47]:
model1.fit(X_train, y_train, nb_epoch=3, batch_size=64)
scores = model1.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/3


13056/32249 [===========>..................] - ETA: 27:32 - loss: 0.6931 - acc: 0.53 - ETA: 20:10 - loss: 0.6896 - acc: 0.67 - ETA: 17:13 - loss: 0.6865 - acc: 0.71 - ETA: 15:36 - loss: 0.6815 - acc: 0.74 - ETA: 14:39 - loss: 0.6751 - acc: 0.77 - ETA: 14:00 - loss: 0.6722 - acc: 0.76 - ETA: 13:37 - loss: 0.6647 - acc: 0.76 - ETA: 13:15 - loss: 0.6538 - acc: 0.76 - ETA: 12:58 - loss: 0.6430 - acc: 0.76 - ETA: 12:44 - loss: 0.6177 - acc: 0.77 - ETA: 12:31 - loss: 0.6262 - acc: 0.77 - ETA: 12:21 - loss: 0.6132 - acc: 0.78 - ETA: 12:14 - loss: 0.6006 - acc: 0.78 - ETA: 12:10 - loss: 0.5905 - acc: 0.78 - ETA: 12:04 - loss: 0.5794 - acc: 0.79 - ETA: 11:58 - loss: 0.5711 - acc: 0.79 - ETA: 11:53 - loss: 0.5696 - acc: 0.79 - ETA: 11:48 - loss: 0.5635 - acc: 0.79 - ETA: 11:43 - loss: 0.5601 - acc: 0.79 - ETA: 11:38 - loss: 0.5535 - acc: 0.79 - ETA: 11:34 - loss: 0.5549 - acc: 0.79 - ETA: 11:32 - loss: 0.5503 - acc: 0.79 - ETA: 11:31 - loss: 0.5488 - acc: 0.79 - ETA: 11:27 - loss: 0.5450 - acc: 

26112/32249 [=======================>......] - ETA: 7:00 - loss: 0.3314 - acc: 0.867 - ETA: 6:59 - loss: 0.3314 - acc: 0.867 - ETA: 6:57 - loss: 0.3309 - acc: 0.867 - ETA: 6:56 - loss: 0.3308 - acc: 0.867 - ETA: 6:55 - loss: 0.3301 - acc: 0.868 - ETA: 6:53 - loss: 0.3295 - acc: 0.868 - ETA: 6:52 - loss: 0.3290 - acc: 0.868 - ETA: 6:51 - loss: 0.3286 - acc: 0.868 - ETA: 6:49 - loss: 0.3282 - acc: 0.868 - ETA: 6:48 - loss: 0.3286 - acc: 0.868 - ETA: 6:46 - loss: 0.3280 - acc: 0.868 - ETA: 6:45 - loss: 0.3272 - acc: 0.869 - ETA: 6:44 - loss: 0.3266 - acc: 0.869 - ETA: 6:42 - loss: 0.3260 - acc: 0.869 - ETA: 6:41 - loss: 0.3262 - acc: 0.869 - ETA: 6:40 - loss: 0.3255 - acc: 0.870 - ETA: 6:38 - loss: 0.3248 - acc: 0.870 - ETA: 6:37 - loss: 0.3245 - acc: 0.870 - ETA: 6:36 - loss: 0.3240 - acc: 0.870 - ETA: 6:34 - loss: 0.3234 - acc: 0.870 - ETA: 6:33 - loss: 0.3231 - acc: 0.871 - ETA: 6:31 - loss: 0.3228 - acc: 0.871 - ETA: 6:30 - loss: 0.3220 - acc: 0.871 - ETA: 6:29 - loss: 0.3220 - acc: 0

32249/32249 [==============================] - ETA: 2:15 - loss: 0.2841 - acc: 0.886 - ETA: 2:13 - loss: 0.2841 - acc: 0.885 - ETA: 2:12 - loss: 0.2838 - acc: 0.886 - ETA: 2:11 - loss: 0.2835 - acc: 0.886 - ETA: 2:09 - loss: 0.2835 - acc: 0.886 - ETA: 2:08 - loss: 0.2833 - acc: 0.886 - ETA: 2:06 - loss: 0.2833 - acc: 0.886 - ETA: 2:05 - loss: 0.2834 - acc: 0.886 - ETA: 2:03 - loss: 0.2832 - acc: 0.886 - ETA: 2:02 - loss: 0.2829 - acc: 0.886 - ETA: 2:01 - loss: 0.2830 - acc: 0.886 - ETA: 1:59 - loss: 0.2828 - acc: 0.886 - ETA: 1:58 - loss: 0.2825 - acc: 0.886 - ETA: 1:56 - loss: 0.2822 - acc: 0.886 - ETA: 1:55 - loss: 0.2824 - acc: 0.886 - ETA: 1:54 - loss: 0.2820 - acc: 0.886 - ETA: 1:52 - loss: 0.2819 - acc: 0.886 - ETA: 1:51 - loss: 0.2820 - acc: 0.886 - ETA: 1:49 - loss: 0.2821 - acc: 0.886 - ETA: 1:48 - loss: 0.2819 - acc: 0.887 - ETA: 1:46 - loss: 0.2817 - acc: 0.887 - ETA: 1:45 - loss: 0.2815 - acc: 0.887 - ETA: 1:44 - loss: 0.2811 - acc: 0.887 - ETA: 1:42 - loss: 0.2808 - acc: 0

13056/32249 [===========>..................] - ETA: 11:34 - loss: 0.1565 - acc: 0.93 - ETA: 11:58 - loss: 0.1810 - acc: 0.93 - ETA: 12:05 - loss: 0.2083 - acc: 0.90 - ETA: 11:58 - loss: 0.2051 - acc: 0.90 - ETA: 11:50 - loss: 0.2014 - acc: 0.90 - ETA: 11:48 - loss: 0.1870 - acc: 0.91 - ETA: 11:45 - loss: 0.1896 - acc: 0.91 - ETA: 11:41 - loss: 0.1738 - acc: 0.91 - ETA: 11:39 - loss: 0.1707 - acc: 0.92 - ETA: 11:36 - loss: 0.1796 - acc: 0.92 - ETA: 11:34 - loss: 0.1734 - acc: 0.92 - ETA: 11:32 - loss: 0.1638 - acc: 0.93 - ETA: 11:31 - loss: 0.1574 - acc: 0.93 - ETA: 11:32 - loss: 0.1569 - acc: 0.93 - ETA: 11:31 - loss: 0.1500 - acc: 0.93 - ETA: 11:28 - loss: 0.1481 - acc: 0.93 - ETA: 11:26 - loss: 0.1534 - acc: 0.93 - ETA: 11:24 - loss: 0.1536 - acc: 0.93 - ETA: 11:22 - loss: 0.1562 - acc: 0.93 - ETA: 11:20 - loss: 0.1597 - acc: 0.93 - ETA: 11:18 - loss: 0.1626 - acc: 0.93 - ETA: 11:16 - loss: 0.1639 - acc: 0.93 - ETA: 11:14 - loss: 0.1645 - acc: 0.93 - ETA: 11:13 - loss: 0.1649 - acc: 

26112/32249 [=======================>......] - ETA: 6:46 - loss: 0.1735 - acc: 0.932 - ETA: 6:45 - loss: 0.1737 - acc: 0.932 - ETA: 6:43 - loss: 0.1735 - acc: 0.932 - ETA: 6:42 - loss: 0.1730 - acc: 0.932 - ETA: 6:41 - loss: 0.1733 - acc: 0.932 - ETA: 6:39 - loss: 0.1734 - acc: 0.932 - ETA: 6:38 - loss: 0.1733 - acc: 0.932 - ETA: 6:36 - loss: 0.1729 - acc: 0.932 - ETA: 6:35 - loss: 0.1728 - acc: 0.932 - ETA: 6:34 - loss: 0.1738 - acc: 0.932 - ETA: 6:33 - loss: 0.1741 - acc: 0.932 - ETA: 6:31 - loss: 0.1743 - acc: 0.932 - ETA: 6:30 - loss: 0.1746 - acc: 0.932 - ETA: 6:28 - loss: 0.1749 - acc: 0.932 - ETA: 6:27 - loss: 0.1746 - acc: 0.932 - ETA: 6:26 - loss: 0.1746 - acc: 0.932 - ETA: 6:24 - loss: 0.1751 - acc: 0.932 - ETA: 6:23 - loss: 0.1754 - acc: 0.932 - ETA: 6:21 - loss: 0.1754 - acc: 0.932 - ETA: 6:20 - loss: 0.1754 - acc: 0.932 - ETA: 6:18 - loss: 0.1757 - acc: 0.931 - ETA: 6:17 - loss: 0.1755 - acc: 0.931 - ETA: 6:16 - loss: 0.1752 - acc: 0.931 - ETA: 6:14 - loss: 0.1752 - acc: 0

32249/32249 [==============================] - ETA: 2:09 - loss: 0.1801 - acc: 0.930 - ETA: 2:07 - loss: 0.1802 - acc: 0.930 - ETA: 2:06 - loss: 0.1801 - acc: 0.930 - ETA: 2:05 - loss: 0.1800 - acc: 0.930 - ETA: 2:03 - loss: 0.1800 - acc: 0.930 - ETA: 2:02 - loss: 0.1799 - acc: 0.930 - ETA: 2:01 - loss: 0.1799 - acc: 0.930 - ETA: 1:59 - loss: 0.1797 - acc: 0.930 - ETA: 1:58 - loss: 0.1794 - acc: 0.930 - ETA: 1:57 - loss: 0.1792 - acc: 0.930 - ETA: 1:55 - loss: 0.1796 - acc: 0.930 - ETA: 1:54 - loss: 0.1797 - acc: 0.930 - ETA: 1:53 - loss: 0.1795 - acc: 0.930 - ETA: 1:51 - loss: 0.1797 - acc: 0.930 - ETA: 1:50 - loss: 0.1796 - acc: 0.930 - ETA: 1:48 - loss: 0.1796 - acc: 0.930 - ETA: 1:47 - loss: 0.1794 - acc: 0.930 - ETA: 1:46 - loss: 0.1792 - acc: 0.930 - ETA: 1:44 - loss: 0.1791 - acc: 0.930 - ETA: 1:43 - loss: 0.1792 - acc: 0.930 - ETA: 1:42 - loss: 0.1793 - acc: 0.930 - ETA: 1:40 - loss: 0.1792 - acc: 0.930 - ETA: 1:39 - loss: 0.1793 - acc: 0.930 - ETA: 1:38 - loss: 0.1792 - acc: 0

13056/32249 [===========>..................] - ETA: 11:39 - loss: 0.1433 - acc: 0.90 - ETA: 12:00 - loss: 0.1052 - acc: 0.95 - ETA: 11:55 - loss: 0.1056 - acc: 0.95 - ETA: 11:44 - loss: 0.1006 - acc: 0.96 - ETA: 11:37 - loss: 0.1049 - acc: 0.96 - ETA: 11:33 - loss: 0.1085 - acc: 0.95 - ETA: 11:29 - loss: 0.1237 - acc: 0.95 - ETA: 11:23 - loss: 0.1527 - acc: 0.94 - ETA: 11:21 - loss: 0.1563 - acc: 0.94 - ETA: 11:20 - loss: 0.1570 - acc: 0.94 - ETA: 11:17 - loss: 0.1522 - acc: 0.94 - ETA: 11:15 - loss: 0.1477 - acc: 0.94 - ETA: 11:15 - loss: 0.1463 - acc: 0.94 - ETA: 11:17 - loss: 0.1505 - acc: 0.94 - ETA: 11:15 - loss: 0.1508 - acc: 0.95 - ETA: 11:13 - loss: 0.1514 - acc: 0.94 - ETA: 11:11 - loss: 0.1504 - acc: 0.94 - ETA: 11:09 - loss: 0.1515 - acc: 0.94 - ETA: 11:07 - loss: 0.1521 - acc: 0.94 - ETA: 11:05 - loss: 0.1500 - acc: 0.94 - ETA: 11:03 - loss: 0.1468 - acc: 0.94 - ETA: 11:02 - loss: 0.1434 - acc: 0.94 - ETA: 11:00 - loss: 0.1461 - acc: 0.94 - ETA: 10:58 - loss: 0.1518 - acc: 

26112/32249 [=======================>......] - ETA: 6:58 - loss: 0.1425 - acc: 0.947 - ETA: 6:56 - loss: 0.1422 - acc: 0.947 - ETA: 6:55 - loss: 0.1421 - acc: 0.947 - ETA: 6:54 - loss: 0.1421 - acc: 0.947 - ETA: 6:52 - loss: 0.1420 - acc: 0.947 - ETA: 6:51 - loss: 0.1418 - acc: 0.947 - ETA: 6:50 - loss: 0.1417 - acc: 0.947 - ETA: 6:48 - loss: 0.1419 - acc: 0.947 - ETA: 6:47 - loss: 0.1414 - acc: 0.947 - ETA: 6:45 - loss: 0.1413 - acc: 0.947 - ETA: 6:44 - loss: 0.1414 - acc: 0.947 - ETA: 6:42 - loss: 0.1411 - acc: 0.947 - ETA: 6:41 - loss: 0.1411 - acc: 0.947 - ETA: 6:40 - loss: 0.1413 - acc: 0.947 - ETA: 6:38 - loss: 0.1412 - acc: 0.947 - ETA: 6:37 - loss: 0.1414 - acc: 0.947 - ETA: 6:36 - loss: 0.1414 - acc: 0.947 - ETA: 6:34 - loss: 0.1412 - acc: 0.947 - ETA: 6:33 - loss: 0.1414 - acc: 0.947 - ETA: 6:32 - loss: 0.1414 - acc: 0.947 - ETA: 6:30 - loss: 0.1420 - acc: 0.947 - ETA: 6:29 - loss: 0.1422 - acc: 0.947 - ETA: 6:27 - loss: 0.1421 - acc: 0.947 - ETA: 6:26 - loss: 0.1425 - acc: 0

32249/32249 [==============================] - ETA: 2:13 - loss: 0.1541 - acc: 0.942 - ETA: 2:12 - loss: 0.1542 - acc: 0.942 - ETA: 2:11 - loss: 0.1541 - acc: 0.942 - ETA: 2:09 - loss: 0.1541 - acc: 0.942 - ETA: 2:08 - loss: 0.1541 - acc: 0.942 - ETA: 2:06 - loss: 0.1544 - acc: 0.942 - ETA: 2:05 - loss: 0.1542 - acc: 0.942 - ETA: 2:04 - loss: 0.1542 - acc: 0.942 - ETA: 2:02 - loss: 0.1542 - acc: 0.942 - ETA: 2:01 - loss: 0.1543 - acc: 0.942 - ETA: 1:59 - loss: 0.1543 - acc: 0.942 - ETA: 1:58 - loss: 0.1542 - acc: 0.942 - ETA: 1:57 - loss: 0.1543 - acc: 0.942 - ETA: 1:55 - loss: 0.1546 - acc: 0.942 - ETA: 1:54 - loss: 0.1545 - acc: 0.942 - ETA: 1:52 - loss: 0.1546 - acc: 0.942 - ETA: 1:51 - loss: 0.1545 - acc: 0.942 - ETA: 1:50 - loss: 0.1545 - acc: 0.942 - ETA: 1:48 - loss: 0.1546 - acc: 0.942 - ETA: 1:47 - loss: 0.1547 - acc: 0.942 - ETA: 1:45 - loss: 0.1545 - acc: 0.942 - ETA: 1:44 - loss: 0.1546 - acc: 0.942 - ETA: 1:42 - loss: 0.1544 - acc: 0.942 - ETA: 1:41 - loss: 0.1544 - acc: 0

# Summary

1. Model1 with one LSTM layer ran for 5 epochs go1 accuracy of 91.32%.
2. Model2 with 2 LSTM layers ran for 3 epochs got accuracy of 91.37%.